# Netflix, try to fill the missing values with omdb api and some simple visualizations


Hi all! This is my first code here in kaggle.
I tried to fill the missing values of the Netflix dataset with the omdb api.
The code below doesn't work if you don't get a key in omdb https://www.omdbapi.com/apikey.aspx
If you don't want to run that part of the code you can got to the Part 2 directly

First of all we need to install the omdb package

# Part 1

In [ ]:
#install omdb package and omdbapi
!pip install omdb
!pip install omdbapi

Load the libraries

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from omdbapi.movie_search import GetMovie #you must install omdbapi

In [ ]:
omdb_apikey = 'XXXXXXXX' #you can get an api key here: https://www.omdbapi.com/apikey.aspx

#import the dataset
df = pd.read_csv("../input/netflix-shows/netflix_titles.csv")

#Check the missing values
df.isna().sum()

#Rename some column names to match the key in omdbapi
df.columns = ['how_id', 'type', 'title', 'Director', 'Actors', 'Country', 'date_added', 'release_year' , 'rating', 'duration', 'listed_in', 'description'] 

Create a list of all the missing data to search the omdb id and missing field

In [ ]:
index_x, index_y = np.where(pd.isna(df))
results = np.column_stack([df.index[index_x], df.columns[index_y]])
results = results.tolist()

Use omdb to find and fill the na and insert into the dataset

In [ ]:
for result in results:
    df.loc[result[0],result[1]]
    show = GetMovie(title=df.loc[result[0],'title'], api_key=omdb_apikey)
    if show.get_all_data() == 'Request limit reached!':
        print('Request limit reached!') #Need to run 3 times because the limit of the api
        break
    if show.get_all_data() == 'Movie not found!':
        print(result[0],'Movie not found!')
    else:
        if show.get_data(result[1]).get(result[1]) != 'N/A':
            df.loc[result[0],result[1]] = show.get_data(result[1]).get(result[1])
            print(result[0], result[1],':', show.get_data(result[1]).get(result[1]))
    results.pop(0)

We have a new dataset with a lot of new values founded in omdbapi

In [ ]:
#save the new dataset to new file
df.to_csv("../input/netflix-titles-omdb/netflix_titles_omdb.csv")

# Part 2

I uploaded the new dataset to Kaggle. If you dont want to use the script of ask for an api key on omdb you can run the project from here:

In [ ]:
#load the new dataset and check it with head, info and isna.sum
df = pd.read_csv("../input/netflix-titles-omdb/netflix_titles_omdb.csv")
df.head()
df.info()
df.isna().sum()

In [ ]:
#Drop all the entries with na
df = df[df["rating"].notna()]
df = df[df["date_added"].notna()]
df = df[df["Actors"].notna()]
df = df[df["Country"].notna()]

#drop column director because there is a lot missing and has no use
df=df.drop(columns="Director")

In [ ]:
#Check again if  we have some other missing data
df.isna().sum()

**whichs countries has more shows**

In [ ]:
# whichs countries has more shows
shows= df.Country.value_counts().head(20)
plt.figure(figsize=(8,5))
ax= sns.barplot(shows.values,shows.index)  
ax.set_xlabel("Shows")
ax.set_ylabel("Countries")

**Release years (I slice top 20 years)**

In [ ]:
years= df.release_year.value_counts().nlargest(20)
plt.figure(figsize=(16,6))
plt.xlabel("Year")
sns.barplot(x=years.index ,y= years.values)

Any comments are welcome!